In [1]:
import re
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!cp -r drive/MyDrive/CIFARData .

In [9]:
CIFAR10Data = np.load('CIFARData.npz')['arr_0']
CIFARDataTest = np.load('CIFARDataTest.npz')['arr_0']
CIFAR10DataVal = np.load('CIFARValData.npz')['arr_0']

In [10]:
CIFAR10Data.shape, CIFARDataTest.shape, CIFAR10DataVal.shape

((4, 4500, 32, 32, 3), (4, 1000, 32, 32, 3), (4, 500, 32, 32, 3))

In [13]:
CIFAR10DataFlat = CIFAR10Data.reshape(-1,32,32,3)
CIFARDataTestFlat = CIFARDataTest.reshape(-1,32,32,3)
CIFAR10DataValFlat = CIFAR10DataVal.reshape(-1,32,32,3)
CIFAR10DataFlat.shape, CIFARDataTestFlat.shape, CIFAR10DataValFlat.shape

((18000, 32, 32, 3), (4000, 32, 32, 3), (2000, 32, 32, 3))

In [14]:
hub_path = 'gs://simclr-checkpoints/simclrv2/finetuned_100pct/r50_1x_sk0/hub/'
module = hub.Module(hub_path, trainable=False)

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
def convert_feature(module, sess, data):
    input = tf.convert_to_tensor(data.astype(np.float32))
    feature_vec = sess.run(module(input))
    return feature_vec

In [17]:
CIFAR10DataFeature = convert_feature(module, sess, CIFAR10DataFlat)

In [18]:
CIFAR10DataFeature.shape

(18000, 2048)

In [19]:
CIFARDataTestFeature = convert_feature(module, sess, CIFARDataTestFlat)
CIFARDataTestFeature.shape

(4000, 2048)

In [20]:
CIFAR10DataValFeature = convert_feature(module, sess, CIFAR10DataValFlat)
CIFARDataTestFeature.shape

(4000, 2048)

In [21]:
np.savez('CIFAR10DataFeature.npz', CIFAR10DataFeature)
np.savez('CIFARDataTestFeature.npz', CIFARDataTestFeature)
np.savez('CIFAR10DataValFeature.npz', CIFAR10DataValFeature)

In [24]:
!cp -r ./CIFARData drive/MyDrive/simCLRfeature/